Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [46]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import matplotlib.pyplot as plt

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [ ]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [4]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]                      ### we want the last element equal to the initial 
    assert set(tsp) == set(range(len(CITIES)))    
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

In [ ]:
print(len(DIST_MATRIX))
print(type(DIST_MATRIX))

print(DIST_MATRIX[0:3])

## First Greedy Algorithm

In [ ]:
visited = np.full(len(CITIES), False)

dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])    #### here we are finding the closest city 
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))


logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)" #### this is the last step to close the cycle
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

## Second Greedy Algorithm

nx.find_cycle(G) in the NetworkX library is used to find a cycle in a graph G. A cycle in a graph is a path that starts and ends at the same node, with each edge and node being visited only once (except the start/end node)

In [98]:
def cyclic(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return True
    except:
        return False

In [106]:
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(CITIES)), 2)
]
visited = set()
edges = set()

In [ ]:
shortest = next(_ for _ in sorted(segments, key=lambda e: e[1]))
visited |= shortest[0]
visited

In [ ]:
shortest

In [114]:
edges |= {tuple(shortest[0])}
segments = [s for s in segments if not cyclic(edges | {tuple(s[0])})]

In [ ]:
edges

In [ ]:
segments

In [ ]:
cyclic(edges)

In [ ]:
### segments and shortest
shortest

NEW ALGORITHM MODIFICATION OF THE GREEDY BUT WITH THE MODIFICATION TO AVOID PING_PONG EFFECT

DEBUG:root:step: Giugliano in Campania -> Naples (11.15km)


[({0, 1}, 349.30401144305745), ({0, 2}, 391.0418681947875), ({0, 3}, 383.0204391314121), ({0, 4}, 199.89962990855176), ({0, 5}, 364.0447808729763), ({0, 6}, 338.8096401206489), ({0, 7}, 609.696280667211), ({0, 8}, 690.431902252931), ({0, 9}, 204.4253666122213), ({0, 10}, 183.29551097355267), ({0, 11}, 290.4028573916701), ({0, 12}, 136.70372473443274), ({0, 13}, 377.57973448944483), ({0, 14}, 301.99049212290043), ({0, 15}, 241.85490798374872), ({0, 16}, 258.5284939080245), ({0, 17}, 625.0755286998198), ({0, 18}, 401.9399754108604), ({0, 19}, 237.16017652663953), ({0, 20}, 401.4539129529872), ({0, 21}, 312.00387878900796), ({0, 22}, 438.9328915074273), ({0, 23}, 239.268808375456), ({0, 24}, 608.6754823442737), ({0, 25}, 287.5153079129395), ({0, 26}, 105.17159317964676), ({0, 27}, 139.23847120123253), ({0, 28}, 345.1318096169963), ({0, 29}, 196.77126979341847), ({0, 30}, 137.84822902835992), ({0, 31}, 635.9878049331178), ({0, 32}, 260.6010504730683), ({0, 33}, 90.5967461797061), ({0, 34},

WORKING BUT WORST THEN THE GREEDY MAYBE STARTING POINT VERY 

In [302]:
visited = np.full(len(CITIES), False)

dist = segments.copy()
print(dist)
iterator = iter(sorted(shortest[0]))
city = next(iterator)
print(city)
next_city = next(iterator)
print(next_city)
visited[city] = True
tsp = list()
tsp.append(int(city))



previous_point= shortest
print("first point " , previous_point)
i=0
logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[next_city,'name']} ({DIST_MATRIX[city,next_city]:.2f}km)"
    )
#for i in range(4):
while not np.all(visited):
    #print()
    #print(i) 
    #print(previous_point)

    dist = [ distance for distance in dist if not all(previous_point[0] in distance for item in dist) ] ### remove previous point 
    
    pick_next_city = []
    for edge in dist:
        tuple_set= edge[0]
        if(next_city in tuple_set ):              ###### we save the tuple in which there are the next city x -> y
            pick_next_city.append(edge)           ###### filtering only the points with the next city 

    pick_next_city=sorted(pick_next_city, key=lambda x: x[1])
    #print(pick_next_city)
    if(len(pick_next_city)==0):
        visited[next_city] = True
        tsp.append(int(next_city))
        #print(visited)
        break
    
    boolean_flag=False
    for element in pick_next_city:
        #print()
        tupla = element[0]
        distance= element[1]    
        #print("element printed " , element)
        filtered_values = {value for value in tupla if value != next_city}.pop()  ### we are extracting the value from the set that are not included
                                                                                  ###    (y , z) we extract z 
        extract = [ distance for distance in dist if all( {filtered_values, city} in distance for item in dist  )  ].pop() # distance STARTING POINT - NEW POINT WITHOUT MIDDLE
                                                                                                                            ### we save the tuple (x, z)
        #print(filtered_values , "   " , city)
        #print("extracted mean (x, z)" , extract)
        #print("comparison between " , previous_point, extract)

        if(extract[1]>previous_point[1]):             #### comparison of distance of  ( x, z ) > (x , y )
            boolean_flag=True
            dist = [ distance for distance in dist if city not in distance[0]   ]      #clean the distance- we are removing all starting point

            #print("the length of dist is ",len(dist))
            previous_point= element
            #print(previous_point)
            #print("the new point is " ,previous_point)
            city= next_city
            #print("the city is ", city)

            next_city =  filtered_values
            #print("the next is",next_city)
            break
        if(boolean_flag==False):
            print("ERROR LOOP IN")
            #print(city , next_city)
    #i+=1
    visited[next_city] = True
    tsp.append(int(city))
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[next_city,'name']} ({DIST_MATRIX[city,next_city]:.2f}km)"
    )

logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)" #### this is the last step to close the cycle
    )
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

DEBUG:root:step: Giugliano in Campania -> Naples (11.15km)
DEBUG:root:step: Naples -> Salerno (46.25km)
DEBUG:root:step: Salerno -> Foggia (109.52km)
DEBUG:root:step: Foggia -> Andria (67.42km)
DEBUG:root:step: Andria -> Bari (50.18km)
DEBUG:root:step: Bari -> Taranto (77.63km)
DEBUG:root:step: Taranto -> Messina (293.02km)
DEBUG:root:step: Messina -> Reggio di Calabria (12.48km)
DEBUG:root:step: Reggio di Calabria -> Catania (84.28km)
DEBUG:root:step: Catania -> Syracuse (51.23km)
DEBUG:root:step: Syracuse -> Palermo (206.47km)
DEBUG:root:step: Palermo -> Latina (374.12km)
DEBUG:root:step: Latina -> Rome (56.84km)
DEBUG:root:step: Rome -> Terni (76.54km)
DEBUG:root:step: Terni -> Perugia (63.64km)
DEBUG:root:step: Perugia -> Ancona (105.17km)
DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Bologna (43.43km)
DEBUG:root:step: Bologn

[({0, 1}, 349.30401144305745), ({0, 2}, 391.0418681947875), ({0, 3}, 383.0204391314121), ({0, 4}, 199.89962990855176), ({0, 5}, 364.0447808729763), ({0, 6}, 338.8096401206489), ({0, 7}, 609.696280667211), ({0, 8}, 690.431902252931), ({0, 9}, 204.4253666122213), ({0, 10}, 183.29551097355267), ({0, 11}, 290.4028573916701), ({0, 12}, 136.70372473443274), ({0, 13}, 377.57973448944483), ({0, 14}, 301.99049212290043), ({0, 15}, 241.85490798374872), ({0, 16}, 258.5284939080245), ({0, 17}, 625.0755286998198), ({0, 18}, 401.9399754108604), ({0, 19}, 237.16017652663953), ({0, 20}, 401.4539129529872), ({0, 21}, 312.00387878900796), ({0, 22}, 438.9328915074273), ({0, 23}, 239.268808375456), ({0, 24}, 608.6754823442737), ({0, 25}, 287.5153079129395), ({0, 26}, 105.17159317964676), ({0, 27}, 139.23847120123253), ({0, 28}, 345.1318096169963), ({0, 29}, 196.77126979341847), ({0, 30}, 137.84822902835992), ({0, 31}, 635.9878049331178), ({0, 32}, 260.6010504730683), ({0, 33}, 90.5967461797061), ({0, 34},

## First Greedy Algorithm

In [301]:
visited = np.full(len(CITIES), False)

dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])    #### here we are finding the closest city 
    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)"
    )
    visited[closest] = True
    city = closest
    tsp.append(int(city))


logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)" #### this is the last step to close the cycle
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")

DEBUG:root:step: Ancona -> Rimini (90.60km)
DEBUG:root:step: Rimini -> Forlì (46.72km)
DEBUG:root:step: Forlì -> Ravenna (26.46km)
DEBUG:root:step: Ravenna -> Ferrara (66.67km)
DEBUG:root:step: Ferrara -> Bologna (43.43km)
DEBUG:root:step: Bologna -> Modena (37.29km)
DEBUG:root:step: Modena -> Reggio nell'Emilia (23.94km)
DEBUG:root:step: Reggio nell'Emilia -> Parma (26.94km)
DEBUG:root:step: Parma -> Piacenza (57.65km)
DEBUG:root:step: Piacenza -> Milan (60.65km)
DEBUG:root:step: Milan -> Monza (14.51km)
DEBUG:root:step: Monza -> Bergamo (33.92km)
DEBUG:root:step: Bergamo -> Brescia (46.02km)
DEBUG:root:step: Brescia -> Verona (61.42km)
DEBUG:root:step: Verona -> Vicenza (44.70km)
DEBUG:root:step: Vicenza -> Padua (30.13km)
DEBUG:root:step: Padua -> Venice (36.07km)
DEBUG:root:step: Venice -> Trieste (115.09km)
DEBUG:root:step: Trieste -> Bolzano (209.68km)
DEBUG:root:step: Bolzano -> Trento (49.94km)
DEBUG:root:step: Trento -> Novara (206.69km)
DEBUG:root:step: Novara -> Turin (84.46

NOT OPTIMIZE BUT WORKING

In [ ]:
visited = np.full(len(CITIES), False)
G = nx.Graph()
dist = segments.copy()
city = next(iter(sorted(shortest[0])))
visited[city] = True

tsp = list()
tsp.append(int(city))


while not np.all(visited):
#for i in range(1):
    ### here we decide how to pick the next 
    picking_next = [  tup  for tup in dist if city in tup[0]  ]
    #print( picking_next )
    ### computing some statistical measure
    distances = [tup[1]  for tup in dist  ]
    mean = np.mean(distances)
    variance = np.var(distances, ddof=0) 

    rank_obj = []
    for index_value in  picking_next:
        tuple_index = index_value[0]
        distance= index_value[1]
        if( distance > mean):
            rank_obj.append(index_value)
    
    if(len(rank_obj)!=0):
        rank_obj=sorted(rank_obj, key=lambda x: x[1])
        next_city = rank_obj[0]
    else:
        if(len(picking_next)!=1):
            for index_value in  picking_next:
                tuple_index = index_value[0]
                distance= index_value[1]
                print(distance , mean)
                if( distance < mean):
                    rank_obj.append(index_value)
            rank_obj = sorted(rank_obj, key=lambda x: x[1])
            if(len(rank_obj)!=1):
                next_city = rank_obj[0]
        else:
            next_city = picking_next[0]

    for indexis in next_city[0]:
        if(indexis!=city):
            feet= indexis
    dist = [  tup  for tup in dist if city not in tup[0]       ] 
    #closest = np.argmin(dist[city])    #### here we are finding the closest city 


    G.add_nodes_from([city, feet])
    G.add_edge(city, feet, weight=round(next_city[1], 2) )



    logging.debug(
        f"step: {CITIES.at[city,'name']} -> {CITIES.at[feet,'name']} ({DIST_MATRIX[city,feet]:.2f}km)"
    )
    visited[feet] = True
    city = feet
    tsp.append(int(city))


logging.debug(
    f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)" #### this is the last step to close the cycle
)
tsp.append(tsp[0])


logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")    # assertion error

In [ ]:
# Define the position of nodes for a simple layout
pos = nx.spring_layout(G)


# Increase figure size
plt.figure(figsize=(10, 6))  # Width=10, Height=6
# Draw the nodes and edges
nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=500, font_size=15)

# Draw edge labels (weights)
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Display the graph
plt.show()

HOW TO CREATE THE GENERAL GRAPH AND REPRESENT THE SEARCH SPACE

In [ ]:
# Create an undirected graph
G = nx.Graph()



for element in segments:
    print()
    print(element)
    # Add nodes
    iterator = iter(element[0])
    A=next(iterator)
    B=next(iterator)    
    G.add_nodes_from([A, B])
    # Add an undirected edge with a weight

    G.add_edge(A, B, weight=round(element[1], 2) )

# Check the edge with weight
print("Edges with weights:", G.edges(data=True))

THIS IS THE SEARCH SPACE

In [ ]:
# Define the position of nodes for a simple layout
pos = nx.spring_layout(G)


# Increase figure size
plt.figure(figsize=(50, 60))  # Width=10, Height=6
# Draw the nodes and edges
nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=500, font_size=15)

# Draw edge labels (weights)
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Display the graph
plt.show()